In [ ]:
import requests
import pandas as pd

api_url = "https://ows.goszakup.gov.kz/v3/refs/ref_kato"
token = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
file = "kato_data.csv"

def retrieve_kato_data():
    """Получает данные КАТО через API и сохраняет в CSV."""
    kato_list = []
    search_after = None

    while True:
        try:
            params = {"limit": 500}
            if search_after:
                params["page"] = "next"
                params["search_after"] = search_after

            response = requests.get(api_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            items = data.get("items", [])
            if not items:
                print("Все данные загружены.")
                break

            for item in items:
                kato_list.append({
                    "code": item.get("code"),
                    "parent_code": item.get("parent_code"),
                    "name_ru": item.get("name_ru"),
                    "name_kz": item.get("name_kz"),
                    "full_name_ru": item.get("full_name_ru"),
                    "full_name_kz": item.get("full_name_kz"),
                    "level": item.get("level_"),
                    "hij" : item.get("hij"),
                    "k" : item.get("k"),
                    "ab" : item.get("ab"),
                    "ef" : item.get("ef"),
                    "cd" : item.get("cd")
                })

            print(f"Текущая загрузка: {len(kato_list)} записей")
            search_after = items[-1].get("code")
            if not data.get("next_page"):
                break

        except requests.exceptions.RequestException as e:
            print(f"Ошибка: {e}")
            break

    export_to_csv(kato_list)

def export_to_csv(dataset):
    """Сохраняет данные в CSV."""
    if not dataset:
        print("Нет данных для сохранения.")
        return
    df = pd.DataFrame(dataset)
    df.to_csv(file, index=False, encoding="utf-8-sig", sep="|", quotechar="'", escapechar="\\")
    print(f"Данные успешно сохранены в файл: {file} ({len(dataset)} записей)")

retrieve_kato_data()